In [1]:
# imports
%matplotlib inline
import seaborn as sns
from pyrt.data.data_trots import *
from pyrt.optimization.vmat import *
from pyrt.tools import *


In [2]:
# Troy Inputs - Prostate
# Find work directory
# cwd = '/Users/troy/Dropbox/CAP Group/TROTS/Prostate_VMAT/'
working_directory = 't-roy/'
cwd = '/media/troy/DataDrive/Dropbox/CAP Group/TROTS/Prostate_VMAT/'

# Load patient information
filename = 'Prostate_VMAT_101.mat'


'''
TEST WHY 100 PTV DOESN'T WORK!!!
'''

back_proj_list_of_dicts = [
   {'PTV':30.,'Bladder':15.,'default':0., 'threshold':0.},
    {'PTV':30.,'Rectum':15.,'default':0., 'threshold':0.},
#     {'PTV':5,'default':1, 'threshold':0.},
    {'PTV':100.,'default':1., 'threshold':0.},
    {'PTV':1.,'default':0., 'threshold':0.}
]
    
# back_proj_list_of_dicts = [
#     {'PTV':10,'default':1, 'threshold':0.}
# ]


vmat_model_params = {
    'target_weights':{'PTV':10000., 'default':500.},
    'oar_weights':{'Rectum':10., 'Bladder':10., 'default':1.},
    'min_intensity':0,
    'max_intensity':1000,
    'cp_redundancy':1, 
    'back_projection_dicts':back_proj_list_of_dicts
}


input_dict = {
    'cwd': cwd,
    'figure_directory':working_directory,
    'filename': filename,
    'Rx': {'PTV': 79.56, 'PTV Vesicles': 72.2},
    'model_params':vmat_model_params
}



In [ ]:
# # Troy Inputs - H&N
# # Find work directory
# # cwd = '/Users/troy/Dropbox/CAP Group/TROTS/Prostate_VMAT/'
# working_directory = 'outputs_troy/'
# cwd = '/media/troy/DataDrive/Dropbox/CAP Group/TROTS/Prostate_VMAT/'

# # Load patient information
# filename = 'Prostate_VMAT_101.mat'

# imrt_model_params = {
#     'target_weights':{'PTV':1000., 'default':500.},
#     'oar_weights':{'Rectum':10., 'Bladder':10., 'default':1.}
# }


# input_dict = {
#     'cwd': cwd,
#     'figure_directory':working_directory,
#     'filename': filename,
#     'Rx': {'PTV': 79.56, 'PTV Vesicles': 72.2},
#     'model_params':imrt_model_params
# }

In [ ]:
model = vmat_mip(input_dict)

In [3]:
model = conformal_arc(input_dict)

---------------------
|  Reading in data  |
---------------------
Building Structures
importing PTV Dij as dense matrix, converting to sparse...
importing PTV Shell 50 mm Dij as dense matrix, converting to sparse...
importing External Ring 20 mm Dij as sparse matrix
importing PTV Shell 5 mm Dij as dense matrix, converting to sparse...
importing PTV Shell 15 mm Dij as dense matrix, converting to sparse...
importing PTV Shell 25 mm Dij as dense matrix, converting to sparse...
importing Rectum Dij as dense matrix, converting to sparse...
importing Anus Dij as dense matrix, converting to sparse...
importing Hip (L) Dij as dense matrix, converting to sparse...
importing Hip (R) Dij as dense matrix, converting to sparse...
importing Bladder Dij as dense matrix, converting to sparse...
importing Patient Dij as dense matrix, converting to sparse...
Building CP
----------------------------
|  Data reading completed  |
----------------------------


In [ ]:
model.optimize(display=5,UB=1000.)


In [ ]:
model.plot_DVH(run_tag='default',saveDVH=True)

In [ ]:
for a in model.apertures:
    plot_aper(a,model.data,str(a.cp_number),save_bool=False,show_plot=True,tight_bool=True, buffer = 2)

In [ ]:
model.aper_intensities


In [ ]:
for a in range(len(model.apertures)):

    print model.apertures[a].intensity = m

In [ ]:

for c in range(model.data.num_control_points):
    
    new_aper = np.zeros(model.data.control_points[c].number_beamlets)
    value, beamlets = pricing_problem_beam_aper(model.data.control_points[c],beamlet_usefulness[model.data.cumulative_beamlets_per_cp[model.data.control_points[c].org_cp_number]:model.data.cumulative_beamlets_per_cp[model.data.control_points[c].org_cp_number+1]])
    if len(beamlets)>0:
        new_aper[np.array(beamlets)] = 1.

        plot_fluence_map(model.data, 
                             model.data.control_points[c],
                             beamlet_usefulness[model.data.cumulative_beamlets_per_cp[model.data.control_points[c].org_cp_number]:model.data.cumulative_beamlets_per_cp[model.data.control_points[c].org_cp_number+1]],
                            tight_bool=False,
                            save_bool=False,
                        show_plot=True)

        plot_fluence_map(model.data, 
                         model.data.control_points[c],
                         new_aper,
                        tight_bool=False,
                        save_bool=False,
                        show_plot=True)
    

In [ ]:
for a in model.apertures:
    plot_aper(a,model.data,str(a.cp_number),save_bool=False,show_plot=True,tight_bool=True, buffer = 2)

In [ ]:
dose_weight_dict = {'PTV':100., 'default':1.}

a = aper_gen_given_dose(dose_weight_dict,model.data, model.data.control_points[0])
for s in model.data.structures:
    print s.is_target
plot_aper(a,model.data,'testaper',show_plot=True, save_bool=True,save_name='tester')







index  :  4
num_beamlets  :  1972
name  :  PTV
Dij  :    (0, 0)	6.974e-06
  (0, 1)	6.91257e-06
  (0, 2)	7.24581e-06
  (0, 3)	6.47621e-06
  (0, 4)	7.59358e-06
  (0, 5)	5.70377e-06
  (0, 6)	6.38256e-06
  (0, 7)	7.44963e-06
  (0, 8)	7.18203e-06
  (0, 9)	5.61889e-06
  (0, 10)	6.72685e-06
  (0, 11)	7.84318e-06
  (0, 12)	6.18718e-06
  (0, 13)	6.47188e-06
  (0, 14)	8.09938e-06
  (0, 15)	7.58084e-06
  (0, 16)	7.57426e-06
  (0, 17)	5.77946e-06
  (0, 18)	6.06533e-06
  (0, 19)	8.62548e-06
  (0, 20)	6.71815e-06
  (0, 21)	7.01317e-06
  (0, 22)	5.69882e-06
  (0, 23)	7.76794e-06
  (0, 24)	6.53823e-06
  :	:
  (1971, 4885)	0.000136643
  (1971, 4886)	4.53583e-05
  (1971, 4887)	4.69558e-05
  (1971, 4888)	7.61416e-05
  (1971, 4889)	2.23783e-05
  (1971, 4890)	0.000131043
  (1971, 4891)	2.26962e-05
  (1971, 4892)	7.88047e-05
  (1971, 4893)	4.55058e-05
  (1971, 4894)	8.41701e-05
  (1971, 4895)	3.24966e-05
  (1971, 4896)	0.00013362
  (1971, 4897)	0.000111896
  (1971, 4898)	3.28224e-05
  (1971, 4899)	4.09229e-